In [1]:
import os
os.chdir('/home/megatron/work')

import megatron
import pandas as pd
import numpy as np
import pickle

In [2]:
image_data = {'image': np.random.random((1000, 48, 48, 3))}
image_node = megatron.nodes.Input('image', (48, 48, 3))

In [3]:
out = megatron.layers.RGBtoGrey()(image_node, 'grey_image')

In [4]:
pipeline = megatron.Pipeline(image_node, out, 'testing_images', version=0.1)

In [5]:
grey_images = pipeline.transform(image_data)

In [7]:
grey_images['grey_image'].shape

(1000, 48, 48)

In [8]:
pipeline.storage.read(lookup='0')['grey_image'].shape

AttributeError: 'NoneType' object has no attribute 'read'